In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import scipy.stats

from utils import (
    detrend, monthly_anomalies, interp_coords, reg_slope
)

In [2]:
t_grid_file = xr.open_zarr('data/ORCA025.L46-KFS006_TroPA.zarr/')

In [3]:
eq = dict(lat=0)
eq_band = dict(lat=slice(-5, 5))
nino3_region = dict(lon=slice(210, 270), **eq_band)
nino4_region = dict(lon=slice(160, 210), **eq_band)
tpart= dict(time=slice('1979-01','2019-12'))

## Prepare Data
### Nemo SSH 

In [4]:
ssh_mon_anom = detrend(
    monthly_anomalies(
        t_grid_file.sossheig.sel(**eq_band, **tpart)
    ).compute()
)
ssh_eq = ssh_mon_anom.sel(**eq)

### D20 

In [5]:
d20_mon = monthly_anomalies(
        t_grid_file.d20.sel(**eq_band, **tpart)
).compute()

In [6]:
d20_mon_anom = detrend(d20_mon.where(
        (~d20_mon.isnull()).all("time"),
        0.
    )
).where(
    (~d20_mon.isnull()).all("time")
)

d20_eq = d20_mon_anom.sel(**eq)

### NLTI

In [7]:
slope = reg_slope(d20_mon_anom, ssh_mon_anom, dim="time") 

nlti = (
    ssh_mon_anom - slope * d20_mon_anom
).rename("NLTI")

In [8]:
nlti_n4 = nlti.sel(
    **nino4_region
).mean(['lat','lon']).rename('nlti_n4').compute()

### SST 

In [9]:
sst_mon_anom = monthly_anomalies(
    t_grid_file.sosstsst.sel(**tpart)
)

n3 = sst_mon_anom.sel(
    **nino3_region
).mean(('lon', 'lat')).rename('sst_n3')

n4 = sst_mon_anom.sel(
    **nino4_region
).mean(('lon', 'lat')).rename('sst_n3')

### From Ashok2007 EMI: CP 

In [10]:
a_region = dict(lon=slice(165, 220), lat=slice(-10, 10))
b_region = dict(lon=slice(250, 290), lat=slice(-15, 5))
c_region = dict(lon=slice(125, 145), lat=slice(-10, 20))

In [11]:
aindex = sst_mon_anom.sel(**a_region).mean(('lon', 'lat'))
bindex = sst_mon_anom.sel(**b_region).mean(('lon', 'lat'))
cindex = sst_mon_anom.sel(**c_region).mean(('lon', 'lat'))
emi = aindex - 0.5 * bindex - 0.5 * cindex

### From Ren and Jin(2010) ict:EP / iwp:CP 

In [12]:
aa = n3 * n4
alpha = np.where(aa > 0, 0.4, 0)
Nwp = n4 - alpha * n3

## 2.detrend and correlation

In [13]:
ds = xr.Dataset(
    {"nlti": nlti_n4, "n4": n4, "emi": emi, "Nwp": Nwp}
).compute()

In [14]:
ds_dt = detrend(ds)

## Write output

In [15]:
!rm -rf data_fig4.zarr
ds_dt.to_zarr('data_fig4.zarr/');